# N-raster

Denne notebook-en viser hvordan man kan laste ned en GeoTIFF fra N-serien, for så å konvertere den til COG.copyright

Eksempelet bruker `Basisdata_3_Celle_25833_N1000Raster_TIFF`, siden denne kun dekker en liten bit av Norge og der mesteparten er hav. Størrelsen er ca. 4.7 Mb.

![Basisdata_3_Celle_25833_N1000Raster_TIFF](img/bilde-av-liten-geotiff.png)

En liten utfordring med N-serien er at NODATA er satt til RGB `255,255,252`. MapServer kan filtrere NODATA med en unik verdi i hvert fargebånd, mens det virker som om QGIS helst vil ha en verdi, som den bruker på alle båndene. Siden bildene er 8-bit, må en av fargene ofres for NODATA. Dette er som oftest helt svart (`0,0,0`) eller helt hvit (`255,255,255`).

I dette eksempelet erstatter jeg alle verdier som har `0` med `1` og så setter jeg NODATA til `0`.

## Last ned fra Geonorge

Geonorge legger neste alle distribuerte filer i et zip-arkiv. Last ned filen og pakk ut GeoTIFF.

In [8]:
from urllib.request import urlretrieve
from zipfile import ZipFile

urlretrieve(
    url="https://nedlasting.geonorge.no/geonorge/Basisdata/N1000Raster/TIFF/Basisdata_3_Celle_25833_N1000Raster_TIFF.zip",
    filename="./data/raster.zip",
)

with ZipFile("./data/raster.zip", "r") as zf:
    zf.extract("33_N1000raster_3.tif", "./data")

## Konverter til COG

I dette eksempelet tar jeg en kopi av "profile", slik at vi beholder filens originale innstillinger. Men jeg endrer "driver" til å være "COG". NODATA settes til `0`.

In [10]:
import numpy as np
import rasterio

with rasterio.open("./data/33_N1000raster_3.tif") as src:
    data: np.ndarray = src.read()
    profile: dict = src.profile.copy()

profile["driver"] = "COG"
profile["nodata"] = 0

mask = np.zeros(src.shape, int)

# Rødt bånd - NODATA: 255
mask = np.where(data[0] == 255, mask + 1, mask)

# Grønt bånd - NODATA: 255
mask = np.where(data[1] == 255, mask + 1, mask)

# Blått bånd - NODATA: 252
mask = np.where(data[2] == 252, mask + 1, mask)

# Endre alle verdier som er `0` til `1`
data[data == 0] = 1

# Dersom alle bånd hadde sin respektive NODATA, settes verdien til `0`
data = np.where(mask == 3, 0, data)

with rasterio.open("./data/cog.tif", "w", **profile) as f:
    f.write(data)